# Import Python Modules

Import the Python modules needed for this notebook.

In [ ]:
# The Google Earth Engine module
import ee
# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap
# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas
# The datetime module is used to specify the dates
# to search for imagery
import datetime
# The colab module to access data from your google drive
from google.colab import drive

## Install pb_gee_tools

The pb_gee_tools module (https://github.com/remotesensinginfo/pb_gee_tools) is a collection of tools used to make it easier to access datasets on Google Earth Engine.

The code below tests whether the module is available to import and if not will install the module from code repository.

In [ ]:
try:
  import pb_gee_tools
except:
  !git --version
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip --version
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets

# Autheticate with Google Earth Engine

The next stage is to specify the project being used for your analysis and to autheticate with Google Earth Engine.

The first time you run the following cell it will come up with a prompt asking your permission to access your GEE account and project.

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-pb-dev') # <==== Replace this with your own EE project string

# Mount Google Drive

To access files within your Google Drive we need to 'mount it' to this colab environment.

Again, it will come up with a prompt asking your permission.

In [ ]:
drive.mount('/content/drive')

# Customise Inputs

This next section is where you need to specify input variables to control the Sentinel-2 image composite which is created.

You need to specify:

*   The path (on your google drive) to the vector layer specifying your region of interest. The vector layer can be any format that Geopandas is able to read (e.g., Shapefile, GeoJSON, GPKG).
*   The start date (i.e., earlier date) for the image search
*   The end date (i.e., later date) for the image search
*   The output no date value used - should always be 0
*   The output image file name (Note, do not specify the file extention - it will have .tif added when created)
*   The output directory on your google drive where the output image will be saved.



In [ ]:
vec_file = '/content/drive/MyDrive/test_shp/test_roi_poly.shp'
start_date = datetime.datetime(year=2020, month=1, day=1)
end_date = datetime.datetime(year=2020, month=12, day=31)
out_no_data_val = 0.0
out_file_name = "sen2_2020_med_comp"
out_gdrive_dir = "test_gee_outs"

# Processing Stages

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
tile_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
sen2_img_col = pb_gee_tools.datasets.get_sen2_sr_harm_collection(
      aoi=tile_aoi,
      start_date=start_date,
      end_date=end_date,
      cloud_thres=70,
      cld_prb_thres = 50,
      nir_drk_thres = 0.15,
      cld_prj_dist = 1,
      clds_buffer = 50,
)
sen2_median = sen2_img_col.median().unmask(out_no_data_val).clip(tile_aoi).toUint16()


In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west)/2
cnt_pt_y = roi_south + (roi_north - roi_south)/2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Add ESRI Satellite Imagery Basemap
esri_img_url = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
map_obj.add_tile_layer(esri_img_url, name="ESRI Imagery", attribution="ESRI")

# Specify the visualisation options.
vis_params = {"min": 0, "max": 4000, "bands": ["B8", "B11", "B4"]}

# Add the Sentinel-2 Composite
map_obj.addLayer(sen2_median, vis_params, "Sentinel-2")

map_obj